In [4]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd
#from ipysheet import sheet, column, to_dataframe

In [5]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [6]:
def create_Text(value):
    return widgets.Text(
    value=value,
    placeholder='Type something',
    #description='String:',
    disabled=False,
    layout=Layout(width='auto', height='auto')
)

In [13]:
def create_grid(xw,yw):
    global xg
    global yg
    global M
    global V
    global VC
    global iM
    global atomsL
    x = xw + 1
    y = yw + 1
    iM = np.arange(x*y).reshape((x,y))+1
    #print(xg,xw,x)
    matrix_Config(xw,yw)
    xg = xw
    yg = yw
    

    grid = widgets.GridspecLayout(x, y, layout=widgets.Layout(width='auto', height='auto'))#height="100%",width="100%")
    #grid[0,0] = create_Int_Text(M[0,0])
    #grid[1,0] = create_Int_Text(M[1,0])
    
    for i in range(x):#2,x):
        for j in range(y):
            if j == 0 and i == 0:#2:
                grid[i,j] = widgets.Label(value='C\E',align_self='center')#r'\(e=mc^2\)')
            elif i == 0:#2:
                grid[i,j] = widgets.Dropdown(
                                options=atomsL,
                                value=V[j-1],
                                #description='Number:',
                                layout=widgets.Layout(width='auto', height='auto')
                            )#create_Dropdown(V[j-1])
            elif j == 0:
                grid[i,j] = widgets.Text(
                                value=VC[i-1],
                                placeholder='Type something',
                                #description='String:',
                                disabled=False,
                                layout=widgets.Layout(width='auto', height='auto')
                            )
            else:
                grid[i, j] = widgets.FloatText(
                                value=M[i-1,j-1],
                                #description='Any:',
                                disabled=False,
                                layout=widgets.Layout(width='auto', height='auto')
                            )#create_Int_Text(M[i-1,j-1])
    display(grid)           
    #out = widgets.interactive_output(f, {grid})            
    return grid

In [14]:
def event_1(v):
    for child2 in Mi.result.children:
        child2.observe(event_2, names = 'value')
    output.clear_output()
    with output:
        display(M,V)
        #print('caca'+str(v))
    return

def matrix_Config(xw,yw):
    global M
    global V
    global VC
    if xg < xw:
        M = np.vstack((M,np.zeros(yw)))
        VC = np.hstack((VC,['Compuesto {}'.format(VC.size+1)]))
    elif xg > xw:
        M = M[:xw,:]
        VC = VC[:xw]
    elif yg < yw:
        M = np.hstack((M,np.zeros([xw,1])))
        V = np.hstack((V,np.ones(1)))
    elif yg > yw:
        M = M[:,:yw]
        V = V[:yw]
    return

In [15]:
def event_2(v):
    num = ""
    for c in v.owner.layout.grid_area:
        if c.isdigit():
            num = num + c
    if int(num) in iM[:1,1:]:
        V[int(num)-2]=int(v.owner.value)
    elif int(num) in iM[1:,1:]:
        M[np.where(iM[1:,1:]==int(num))]=float(v.owner.value)
    elif int(num) in iM[1:,:1]:
        VC[np.where(iM[1:,:1]==int(num))[0]]=v.owner.value
    output.clear_output()
    with output:
        display(M,V) 
        #print('caca'+str(v),v.owner.layout.grid_area,int(num),v.owner.value)
    return

In [16]:
def peso_muestra(v):
    global P
    P = v['new']
def calcular(obj):
    global P
    global aW
    global M
    global V
    global VC
    #----------Definiciones------------
    #Número de cationes
    Nc = np.size(V)-2
    
    #Matriz de compuestos
    ##_-_-_-_-_-_-_-|B|P|S|L|K|T|C|O|
    ##_-_-_-_-_-_-_-|a|b|r|a|-|i|-|-|
    ##_-_-_-_-_-_-_-|-|-|-|-|-|-|-|-|
    #MC = np.array([[1,0,0,0,0,0,1,3]   #BaCO3
    #              ,[0,1,0,0,0,0,0,1]   #PbO
    #              ,[0,0,1,0,0,0,1,3]   #SrCO3
    #              ,[0,0,0,2,0,0,0,3]   #La2O3
    #              ,[0,0,0,0,2,0,1,3]   #K2CO3
    #              ,[0,0,0,0,0,1,0,2]     #TiO2
    #              ,[1/5,1/5,1/5,1/5,1/5,1,0,3]])     #Ba1/5 Pb1/5 Sr1/5 La1/5 K1/5 Ti O3
    #_-_-_-_-_-_-_-|Ba-|Pb-|Sr-|La-|K--|T|C|O|
    #_-_-_-_-_-_-_-|---|---|---|---|---|i|-|-|
    MC = M
    #sMC = np.shape(MC)

    #Elementos ([#numero atomico])
    #_-_-_-_-_-_-_|Ba|Pb|Sr|La|K-|Ti|C|O|
    #NE = np.array([56,82,38,57,19,22,6,8])
    NE = V.astype(int)

    #Vector de pesos atomicos
    VPa = aW[NE-1,1]

    #Vector de peso molecular
    VPm = MC@VPa

    #----------Estequiometria----------------

    #Matriz molar de reactivos
    #Para obtener 1 mol de un átomo ¿cuantos moles de cada reactivo se necesitan?
    MMr = (1/(MC[0:Nc,0:Nc]+np.ones([Nc,Nc])-np.identity(Nc)))-(np.ones([Nc,Nc])-np.identity(Nc))

    #Vector molar de producto
    VMp = MC[Nc,0:Nc]

    #Vector molar de reactivos balanceados
    VMrb = MMr@VMp

    #Vector de proporción peso reactivo / peso producto

    Vrp = VPm[0:Nc]/VPm[Nc]

    #Cantidad de producto

    #P = 1

    #Pureza
    #_-_-_-_-_-_-_-_-|Ba---|Pb---|Sr---|La---|K-----|Ti---|
    #Assay = np.array([99.98,99.99,99.90,99.90,99.997,99.80])

    #Cantidad de reactivo
    CR = VMrb*Vrp*P
    #Cantidad de reactivo con pureza
    #CRp = VMrb*Vrp*P*(100/Assay)
    DF = pd.DataFrame(CR,index=VC[:np.size(VC)-1],columns=['Masa R. (g)'])
    output.clear_output()
    with output:
        #print('Calcular')
        display(M,V,CR,DF)
    return

In [17]:
aW = np.loadtxt(fname = "atomicWeight.txt",delimiter="\t")
atoms = np.loadtxt(fname = "atoms.txt",delimiter="\t",dtype={'names': ('element', 'number', 'weight'),'formats': ("U12", 'i4', 'f8')})
atomsL = atoms[['element','number']].tolist()
xg=7
yg=8
#M = np.zeros([xg,yg],dtype=np.float64)
M = np.array([[1,0,0,0,0,0,1,3]   #BaCO3
              ,[0,1,0,0,0,0,0,1]   #PbO
              ,[0,0,1,0,0,0,1,3]   #SrCO3
              ,[0,0,0,2,0,0,0,3]   #La2O3
              ,[0,0,0,0,2,0,1,3]   #K2CO3
              ,[0,0,0,0,0,1,0,2]     #TiO2
              ,[1/5,1/5,1/5,1/5,1/5,1,0,3]])     #Ba1/5 Pb1/5 Sr1/5 La1/5 K1/5 Ti O3
#V = np.ones([yg])
V =  np.array([56,82,38,57,19,22,6,8])
#VC =  np.array(['Compuesto {}'.format(i) for i in range(1, xg+1)])
VC = np.array(['BaCO3','PbO','SrCO3','La2O3','K2CO3','TiO2','Ba1/5 Pb1/5 Sr1/5 La1/5 K1/5 Ti O3'])
P = 1

In [18]:
output = widgets.Output()
Mi = widgets.interactive(create_grid,xw=widgets.BoundedIntText(
    value=7,
    min=1,
    max=10,
    step=1,
    description='#Compuestos',
    disabled=False
),yw=widgets.BoundedIntText(
    value=8,
    min=1,
    max=10,
    step=1,
    description='#Elementos',
    disabled=False
))
Pm=widgets.FloatText(
    value=1,
    description='Peso de muestra',
    disabled=False
)
Bc = widgets.Button(
    description='Calcular',
    disabled=False
)

Pm.value = P
Mi.children[0].value=xg
Mi.children[1].value=yg
display(Mi,Pm,Bc,output)

Pm.observe(peso_muestra, names='value')
for child1 in Mi.children:
    child1.observe(event_1, names = 'value')
for child2 in Mi.result.children:
    child2.observe(event_2, names = 'value')
Bc.on_click(calcular)

interactive(children=(BoundedIntText(value=7, description='#Compuestos', max=10, min=1), BoundedIntText(value=…

FloatText(value=1.0, description='Peso de muestra')

Button(description='Calcular', style=ButtonStyle())

Output()

In [21]:
!pip freeze > requirements.txt